In [ ]:
!pip install defeatbeta-api
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 3.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import defeatbeta_api
from defeatbeta_api.data.ticker import Ticker

In [ ]:
def _get_financials(ticker_symbol):
    """
    Fetch financial data and save to an Excel file with automated formatting
    (bold headers, auto column widths, number formats, freeze panes, etc.).
    """

    # 1) Fetch data via your existing library
    ticker = Ticker(ticker_symbol)
    info     = ticker.info()
    officers = ticker.officers()
    summary  = ticker.summary()
    pe       = ticker.ttm_pe()

    q_inc = ticker.quarterly_income_statement().df()
    a_inc = ticker.annual_income_statement().df()

    q_bs = ticker.quarterly_balance_sheet().df()
    a_bs = ticker.annual_balance_sheet().df()

    q_cf = ticker.quarterly_cash_flow().df()
    a_cf = ticker.annual_cash_flow().df()

    earnings  = ticker.earnings()
    dividends = ticker.dividends()

    # 2) Create a dict of (sheet_name, DataFrame)
    sheets = {
        # "Info":           info,
        # "Officers":       officers,
        # "Summary":        summary,
        # "PE":             pe,
        "Q_Income_Stmt":  q_inc,
        "A_Income_Stmt":  a_inc,
        "Q_Balance_Sheet":q_bs,
        "A_Balance_Sheet":a_bs,
        "Q_Cash_Flow":    q_cf,
        "A_Cash_Flow":    a_cf,
        "Earnings":       earnings,
        "Dividends":      dividends
    }

    # 3) Open an ExcelWriter with engine="xlsxwriter"
    output_filename = f"{ticker_symbol}_financials.xlsx"
    with pd.ExcelWriter(output_filename, engine="xlsxwriter") as writer:
        # Grab the XlsxWriter workbook object
        workbook  = writer.book

        # Define some common formats
        header_fmt = workbook.add_format({
            "bold": True,
            "align": "center",
            "valign": "vcenter",
            "bg_color": "#D9E1F2",
            "border": 1
        })
        text_fmt = workbook.add_format({"num_format": "@", "border": 1})
        integer_fmt = workbook.add_format({"num_format": "#,##0", "border": 1})
        float_fmt   = workbook.add_format({"num_format": "#,##0.00", "border": 1})
        date_fmt    = workbook.add_format({"num_format": "yyyy-mm-dd", "border": 1})

        # (Optional) a small function to pick a format based on dtype
        def _select_format(series_dtype):
            if np.issubdtype(series_dtype, np.integer):
                return integer_fmt
            elif np.issubdtype(series_dtype, np.floating):
                return float_fmt
            elif np.issubdtype(series_dtype, np.datetime64):
                return date_fmt
            else:
                return text_fmt

        # 4) Iterate through sheets, write DataFrame, and apply formatting
        for sheet_name, df in sheets.items():
            # Write DataFrame to sheet (without index, or with index if you prefer)
            df.to_excel(writer, sheet_name=sheet_name, index=False, startrow=1, header=False)
            worksheet = writer.sheets[sheet_name]

            # Write the header with the header_fmt
            for col_num, column_title in enumerate(df.columns):
                worksheet.write(0, col_num, column_title, header_fmt)

            # Auto-adjust column widths based on max length in each column
            for idx, col in enumerate(df.columns):
                # Determine max length in this column (including header)
                series = df[col].astype(str)
                max_len = max(
                    series.map(len).max(),
                    len(str(col))
                ) + 2  # add a little extra
                worksheet.set_column(idx, idx, max_len)

            # Apply number/datetime formatting row-by-row
            # Starting at row=1 since row 0 is header
            for col_num, column in enumerate(df.columns):
                col_dtype = df[column].dtype
                cell_format = _select_format(col_dtype)

                # Only apply format if not "object" (i.e., text)
                if not np.issubdtype(col_dtype, np.object_):
                    # We know df has N rows; write formatting for entire column range
                    # from row 1 to row df.shape[0]
                    start_row = 1
                    end_row   = df.shape[0]
                    worksheet.set_column(
                        col_num,
                        col_num,
                        None,  # width was already set above
                        cell_format,
                        {"row": start_row, "rows": end_row}  # apply to all data cells
                    )

            # 5) Freeze the top header row
            worksheet.freeze_panes(1, 0)

        # When exiting the `with` block, writer.save() is called automatically

    print(f"Saved formatted Excel file as: {output_filename}")

In [ ]:
stock_ticker = "GE"
_get_financials(stock_ticker)

Saved formatted Excel file as: NVO_financials.xlsx
